# Audiobooks business case

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set()
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load the data
raw_df= pd.read_csv('Audiobooks_data.csv')

In [87]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14084 entries, 0 to 14083
Data columns (total 12 columns):
ID                  14084 non-null int64
overall_minutes     14084 non-null float64
avg_minutes         14084 non-null int64
price_overall       14084 non-null float64
price_avg           14084 non-null float64
review              14084 non-null int64
review_scale        2468 non-null float64
completion          14084 non-null float64
minutes_listened    14084 non-null float64
support_request     14084 non-null int64
days_between        14084 non-null int64
convert             14084 non-null int64
dtypes: float64(6), int64(6)
memory usage: 1.3 MB


In [88]:
# setting index as ID and fill NaN with zero
raw_df.set_index('ID', inplace = True)
raw_df.review_scale.fillna(0, axis = 0, inplace = True)

In [89]:
raw_df.describe()

,overall_minutes,avg_minutes,price_overall,price_avg,review,review_scale,completion,minutes_listened,support_request,days_between,convert
count,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000
mean,1591.281685,1678.608634,7.103791,7.543805,0.160750,1.561132,0.125659,189.888983,0.070222,61.935033,0.158833
std,504.340663,654.838599,4.931673,5.560129,0.367313,3.447537,0.241206,371.084010,0.472157,88.207634,0.365533
min,216.000000,216.000000,3.860000,3.860000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1188.000000,1188.000000,5.330000,5.330000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1620.000000,1620.000000,5.950000,6.070000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000
75%,2160.000000,2160.000000,8.000000,8.000000,0.000000,0.000000,0.130000,194.400000,0.000000,105.000000,0.000000
max,2160.000000,7020.000000,130.940000,130.940000,1.000000,10.000000,1.000000,2160.000000,30.000000,464.000000,1.000000


### Feature engineering/ selection

In [90]:
# average price per minute to exacerbate the value
raw_df['price_per_minute'] = raw_df.price_avg / raw_df.avg_minutes

In [91]:
# average rating is 8.91, instead of replacing no ratings with mean, any mean greater than 8 will be assigned 1
raw_df['review'] = [1 if rating > 8 else 0 for rating in raw_df['review_scale']]

In [92]:
# get the difference between the two and eliminate the average minutes column 
raw_df['minutes_difference'] = raw_df.overall_minutes - raw_df.avg_minutes 


In [93]:
# get the difference between the two and eliminate the average price column 
raw_df['price_difference'] = raw_df.price_overall - raw_df.price_avg


In [94]:
# drop review scale column due to missing data; alternatively we could 
raw_df.drop(['avg_minutes', 'price_avg', 'review_scale',], axis = 1, inplace = True)

In [95]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14084 entries, 994 to 251
Data columns (total 11 columns):
overall_minutes       14084 non-null float64
price_overall         14084 non-null float64
review                14084 non-null int64
completion            14084 non-null float64
minutes_listened      14084 non-null float64
support_request       14084 non-null int64
days_between          14084 non-null int64
convert               14084 non-null int64
price_per_minute      14084 non-null float64
minutes_difference    14084 non-null float64
price_difference      14084 non-null float64
dtypes: float64(7), int64(4)
memory usage: 1.3 MB


### Preprocessing

In [96]:
# When the data was collected it was actually arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it; first shuffle
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_df = raw_df.sample(frac=1)

In [97]:
# seperating the independent and dependant variables to oversample
inputs = shuffled_df.drop('convert', axis = 1)
targets = shuffled_df.convert

In [98]:
# balance data for better results from sampling 
smote = SMOTE(random_state = 13)
smote_X, smote_y = smote.fit_sample(inputs, targets)
X = pd.DataFrame(smote_X, columns = inputs.columns )
y= pd.DataFrame(smote_y, columns=['convert'])
# # we can Check the numbers of our data
print("length is",len(smote_X))
print("Number of no conversion",len(y[y['convert'] == 0]))
print("Number of conversions",len(y[y['convert'] == 1]))
print("Proportion of no conversion ", len(y[y['convert'] == 0]) / len(X))
print("Proportion of conversion", len(y[y['convert'] == 1]) / len(X))
# class is balanced with equal proportions

length is 23694
Number of no conversion 11847
Number of conversions 11847
Proportion of no conversion  0.5
Proportion of conversion 0.5


In [99]:
# scaling input data can try Minmax, quantile transform for better results
scaler = StandardScaler()
inputs = scaler.fit_transform(X)
inputs_df = pd.DataFrame(inputs, columns = X.columns)
# create one df to seperate into train, val and test npz files
df = inputs_df.merge(y, left_index = True, right_index = True)
df = df.astype('float32')

In [100]:
# Split the dataset into train, validation, and test
# Count the total number of samples
samples_count = df.shape[0]
# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = int(0.1 * samples_count)

# check 
train_samples_count + validation_samples_count + test_samples_count, samples_count

(23693, 23694)

In [102]:
# second shuffle to ensure randomness
df = df.sample(frac = 1)

# Create variables that record the inputs and targets for training.
train_inputs = df.iloc[:train_samples_count ]
train_targets =  df.convert.iloc[:train_samples_count]

# Create variables that record the inputs and targets for validation.
val_inputs = df.iloc[train_samples_count : train_samples_count + validation_samples_count]
val_targets = df.convert.iloc[train_samples_count : train_samples_count + validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = df.iloc[train_samples_count + validation_samples_count:]
test_targets = df.convert.iloc[train_samples_count + validation_samples_count:]

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print('Targets', 'Obs', 'Ratio')
print(np.sum(train_targets), train_samples_count, round(np.sum(train_targets) / train_samples_count, 2))
print(np.sum(val_targets), validation_samples_count, round(np.sum(val_targets) / validation_samples_count, 2))
print(np.sum(test_targets), test_samples_count, round(np.sum(test_targets) / test_samples_count, 2))

Targets Obs Ratio
9402.0 18955 0.5
1217.0 2369 0.51
1228.0 2369 0.52


### Save the three datasets in *.npz

In [104]:
# Saving the three datasets in *.npz. for future use
np.savez('Audiobooks_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_validation', inputs=val_inputs, targets=val_targets)
np.savez('Audiobooks_test', inputs=test_inputs, targets=test_targets)

### Batching

In [105]:
# Create a class that will do the batching for the algorithm
# This code is extremely reusable. You should just change Audiobooks_data everywhere in the code
class Audiobooks_Data_Reader():
    # Dataset is a mandatory arugment, while the batch_size is optional
    # If you don't input batch_size, it will automatically take the value: None
    def __init__(self, dataset, batch_size = None):
    
        # The dataset that loads is one of "train", "validation", "test".
        # e.g. if I call this class with x('train',5), it will load 'Audiobooks_data_train.npz' with a batch size of 5.
        npz = np.load('Audiobooks_data_{0}.npz'.format(dataset))
        
        # Two variables that take the values of the inputs and the targets. Inputs are floats, targets are integers
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        # Counts the batch number, given the size you feed it later
        # If the batch size is None, we are either validating or testing, so we want to take the data in a single batch
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    # A method which loads the next batch
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        # You slice the dataset in batches and then the "next" function loads them one after the other
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        # One-hot encode the targets. In this example it's a bit superfluous since we have a 0/1 column 
        # as a target already but we're giving you the code regardless, as it will be useful for any 
        # classification task with more than one target column
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        # The function will return the inputs batch and the one-hot encoded targets
        return inputs_batch, targets_one_hot
    
        
    # A method needed for iterating over the batches, as we will put them in a loop
    # This tells Python that the class we're defining is iterable, i.e. that we can use it like:
    # for input, output in data: 
        # do things
    # An iterator in Python is a class with a method __next__ that defines exactly how to iterate through its objects
    def __iter__(self):
        return self

# Training 1

In [107]:
# Input size depends on the number of input variables. 
input_size = 10
# Output size is 2, as we one-hot encoded the targets.
output_size = 2
# Choose a hidden_layer_size
hidden_layer = [350, 450, 500, 700]
for hidden_layer_size in hidden_layer:
    # Reset the default graph, so you can fiddle with the hyperparameters and then rerun the code.
    tf.reset_default_graph()

    # Create the placeholders
    inputs = tf.placeholder(tf.float32, [None, input_size])
    targets = tf.placeholder(tf.int32, [None, output_size])

    # Outline the model. We will create a net with 2 hidden layers
    weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
    biases_1 = tf.get_variable("biases_1", [hidden_layer_size])
    outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

    weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
    biases_2 = tf.get_variable("biases_2", [hidden_layer_size])
    outputs_2 = tf.nn.sigmoid(tf.matmul(outputs_1, weights_2) + biases_2)

    weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
    biases_3 = tf.get_variable("biases_3", [output_size])
    # We will incorporate the softmax activation into the loss, as in the previous example
    outputs = tf.matmul(outputs_2, weights_3) + biases_3

    # Use the softmax cross entropy loss with logits
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)
    mean_loss = tf.reduce_mean(loss)

    # Get a 0 or 1 for every input indicating whether it output the correct answer
    out_equals_target = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))
    accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

    # Optimize with Adam
    optimize = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(mean_loss)

    # Create a session
    sess = tf.InteractiveSession()

    # Initialize the variables
    initializer = tf.global_variables_initializer()
    sess.run(initializer)

    # Choose the batch size
    batch_size = 100

    # Set early stopping mechanisms
    max_epochs = 100
    prev_validation_loss = 9999999.

    # Load the first batch of training and validation, using the class we created. 
    # Arguments are ending of 'Audiobooks_Data_<...>', where for <...> we input 'train', 'validation', or 'test'
    # depending on what we want to load
    train_data = Audiobooks_Data_Reader('train', batch_size)
    validation_data = Audiobooks_Data_Reader('validation')

    # Create the loop for epochs 
    for epoch_counter in range(max_epochs):

        # Set the epoch loss to 0, and make it a float
        curr_epoch_loss = 0.

        # Iterate over the training data 
        # Since train_data is an instance of the Audiobooks_Data_Reader class,
        # we can iterate through it by implicitly using the __next__ method we defined above.
        # As a reminder, it batches samples together, one-hot encodes the targets, and returns
        # inputs and targets batch by batch
        for input_batch, target_batch in train_data:
            _, batch_loss = sess.run([optimize, mean_loss], 
                feed_dict={inputs: input_batch, targets: target_batch})

            #Record the batch loss into the current epoch loss
            curr_epoch_loss += batch_loss

        # Find the mean curr_epoch_loss
        # batch_count is a variable, defined in the Audiobooks_Data_Reader class
        curr_epoch_loss /= train_data.batch_count

        # Set validation loss and accuracy for the epoch to zero
        validation_loss = 0.
        validation_accuracy = 0.

        # Use the same logic of the code to forward propagate the validation set
        # There will be a single batch, as the class was created in this way
        for input_batch, target_batch in validation_data:
            validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                feed_dict={inputs: input_batch, targets: target_batch})

        # Print statistics for the current epoch
        print('Epoch '+str(epoch_counter+1)+
              '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
              '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
              '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%. '+
             'Hidden_layers:' '{0}'.format(hidden_layer_size))

        # Trigger early stopping if validation loss begins increasing.
        if validation_loss > prev_validation_loss:
            break

        # Store this epoch's validation loss to be used as previous in the next iteration.
        prev_validation_loss = validation_loss

    print('End of training.')

Epoch 1. Training loss: 0.728. Validation loss: 0.638. Validation accuracy: 69.13%. Hidden_layers:350
Epoch 2. Training loss: 0.611. Validation loss: 0.577. Validation accuracy: 74.94%. Hidden_layers:350
Epoch 3. Training loss: 0.563. Validation loss: 0.537. Validation accuracy: 74.05%. Hidden_layers:350
Epoch 4. Training loss: 0.529. Validation loss: 0.507. Validation accuracy: 75.62%. Hidden_layers:350
Epoch 5. Training loss: 0.502. Validation loss: 0.483. Validation accuracy: 75.39%. Hidden_layers:350
Epoch 6. Training loss: 0.481. Validation loss: 0.465. Validation accuracy: 75.39%. Hidden_layers:350
Epoch 7. Training loss: 0.464. Validation loss: 0.450. Validation accuracy: 75.84%. Hidden_layers:350
Epoch 8. Training loss: 0.450. Validation loss: 0.438. Validation accuracy: 75.62%. Hidden_layers:350
Epoch 9. Training loss: 0.439. Validation loss: 0.428. Validation accuracy: 75.62%. Hidden_layers:350
Epoch 10. Training loss: 0.429. Validation loss: 0.420. Validation accuracy: 75.17

Epoch 81. Training loss: 0.323. Validation loss: 0.341. Validation accuracy: 79.64%. Hidden_layers:350
Epoch 82. Training loss: 0.323. Validation loss: 0.341. Validation accuracy: 79.64%. Hidden_layers:350
Epoch 83. Training loss: 0.323. Validation loss: 0.341. Validation accuracy: 79.42%. Hidden_layers:350
Epoch 84. Training loss: 0.322. Validation loss: 0.341. Validation accuracy: 79.42%. Hidden_layers:350
Epoch 85. Training loss: 0.322. Validation loss: 0.340. Validation accuracy: 79.42%. Hidden_layers:350
Epoch 86. Training loss: 0.322. Validation loss: 0.340. Validation accuracy: 79.42%. Hidden_layers:350
Epoch 87. Training loss: 0.322. Validation loss: 0.340. Validation accuracy: 79.42%. Hidden_layers:350
Epoch 88. Training loss: 0.321. Validation loss: 0.340. Validation accuracy: 79.42%. Hidden_layers:350
Epoch 89. Training loss: 0.321. Validation loss: 0.340. Validation accuracy: 79.42%. Hidden_layers:350
Epoch 90. Training loss: 0.321. Validation loss: 0.339. Validation accura

Epoch 60. Training loss: 0.326. Validation loss: 0.344. Validation accuracy: 79.42%. Hidden_layers:450
Epoch 61. Training loss: 0.326. Validation loss: 0.344. Validation accuracy: 79.42%. Hidden_layers:450
Epoch 62. Training loss: 0.325. Validation loss: 0.343. Validation accuracy: 79.42%. Hidden_layers:450
Epoch 63. Training loss: 0.325. Validation loss: 0.343. Validation accuracy: 79.42%. Hidden_layers:450
Epoch 64. Training loss: 0.324. Validation loss: 0.343. Validation accuracy: 79.42%. Hidden_layers:450
Epoch 65. Training loss: 0.324. Validation loss: 0.342. Validation accuracy: 79.42%. Hidden_layers:450
Epoch 66. Training loss: 0.324. Validation loss: 0.342. Validation accuracy: 79.64%. Hidden_layers:450
Epoch 67. Training loss: 0.323. Validation loss: 0.342. Validation accuracy: 79.42%. Hidden_layers:450
Epoch 68. Training loss: 0.323. Validation loss: 0.342. Validation accuracy: 79.64%. Hidden_layers:450
Epoch 69. Training loss: 0.323. Validation loss: 0.341. Validation accura

Epoch 40. Training loss: 0.336. Validation loss: 0.350. Validation accuracy: 78.75%. Hidden_layers:500
Epoch 41. Training loss: 0.335. Validation loss: 0.350. Validation accuracy: 78.75%. Hidden_layers:500
Epoch 42. Training loss: 0.335. Validation loss: 0.349. Validation accuracy: 78.75%. Hidden_layers:500
Epoch 43. Training loss: 0.334. Validation loss: 0.349. Validation accuracy: 78.75%. Hidden_layers:500
Epoch 44. Training loss: 0.333. Validation loss: 0.349. Validation accuracy: 78.75%. Hidden_layers:500
Epoch 45. Training loss: 0.333. Validation loss: 0.348. Validation accuracy: 79.19%. Hidden_layers:500
Epoch 46. Training loss: 0.332. Validation loss: 0.348. Validation accuracy: 79.19%. Hidden_layers:500
Epoch 47. Training loss: 0.331. Validation loss: 0.347. Validation accuracy: 79.42%. Hidden_layers:500
Epoch 48. Training loss: 0.331. Validation loss: 0.347. Validation accuracy: 79.64%. Hidden_layers:500
Epoch 49. Training loss: 0.330. Validation loss: 0.346. Validation accura

Epoch 20. Training loss: 0.353. Validation loss: 0.361. Validation accuracy: 78.30%. Hidden_layers:700
Epoch 21. Training loss: 0.351. Validation loss: 0.360. Validation accuracy: 78.30%. Hidden_layers:700
Epoch 22. Training loss: 0.349. Validation loss: 0.359. Validation accuracy: 78.30%. Hidden_layers:700
Epoch 23. Training loss: 0.347. Validation loss: 0.358. Validation accuracy: 78.30%. Hidden_layers:700
Epoch 24. Training loss: 0.346. Validation loss: 0.357. Validation accuracy: 78.30%. Hidden_layers:700
Epoch 25. Training loss: 0.344. Validation loss: 0.356. Validation accuracy: 78.52%. Hidden_layers:700
Epoch 26. Training loss: 0.343. Validation loss: 0.355. Validation accuracy: 78.75%. Hidden_layers:700
Epoch 27. Training loss: 0.342. Validation loss: 0.354. Validation accuracy: 78.97%. Hidden_layers:700
Epoch 28. Training loss: 0.340. Validation loss: 0.354. Validation accuracy: 79.19%. Hidden_layers:700
Epoch 29. Training loss: 0.339. Validation loss: 0.353. Validation accura

Epoch 100. Training loss: 0.310. Validation loss: 0.333. Validation accuracy: 81.43%. Hidden_layers:700
End of training.


### Testing 1

In [108]:
# Load the test data, following the same logic as we did for the train_data and validation data
test_data = Audiobooks_Data_Reader('test')

# Forward propagate through the training set. This time we only need the accuracy
for inputs_batch, targets_batch in test_data:
    test_accuracy = sess.run([accuracy],
                     feed_dict={inputs: inputs_batch, targets: targets_batch})

# Get the test accuracy in percentages
# When sess.run is has a single output, we get a list (that's how it was coded by Google), rather than a float.
# Therefore, we must take the first value from the list (the value at position 0)
test_accuracy_percent = test_accuracy[0] * 100.

# Print the test accuracy
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 83.93%


### Training 2

In [151]:
from tensorflow.contrib.opt import NadamOptimizer

# Input size depends on the number of input variables. 
input_size = 10
# Output size is 2, as we one-hot encoded the targets.
output_size = 2
# Choose a hidden_layer_size
hidden_layer = [700]
for hidden_layer_size in hidden_layer:
    # Reset the default graph, so you can fiddle with the hyperparameters and then rerun the code.
    tf.reset_default_graph()

    # Create the placeholders
    inputs = tf.placeholder(tf.float32, [None, input_size])
    targets = tf.placeholder(tf.int32, [None, output_size])

    # Outline the model. We will create a net with 2 hidden layers
    weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
    biases_1 = tf.get_variable("biases_1", [hidden_layer_size])
    outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

    weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
    biases_2 = tf.get_variable("biases_2", [hidden_layer_size])
    outputs_2 = tf.nn.sigmoid(tf.matmul(outputs_1, weights_2) + biases_2)

    weights_3 = tf.get_variable("weights_3", [hidden_layer_size, hidden_layer_size])
    biases_3 = tf.get_variable("biases_3", [hidden_layer_size])
    # We will incorporate the softmax activation into the loss, as in the previous example
    outputs_3 = tf.nn.leaky_relu(tf.matmul(outputs_2, weights_3) + biases_3)

    weights_4 = tf.get_variable("weights_4", [hidden_layer_size, hidden_layer_size])
    biases_4 = tf.get_variable("biases_4", [hidden_layer_size])
    # We will incorporate the softmax activation into the loss, as in the previous example
    outputs_4 = tf.nn.sigmoid(tf.matmul(outputs_3, weights_4) + biases_4)
    
    weights_5 = tf.get_variable("weights_5", [hidden_layer_size, output_size])
    biases_5 = tf.get_variable("biases_5", [output_size])
    # We will incorporate the softmax activation into the loss, as in the previous example
    outputs = tf.matmul(outputs_4, weights_5) + biases_5
    
    # Use the softmax cross entropy loss with logits
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)
    mean_loss = tf.reduce_mean(loss)

    # Get a 0 or 1 for every input indicating whether it output the correct answer
    out_equals_target = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))
    accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

    # Optimize with Adam
    optimize = NadamOptimizer(learning_rate=0.0001).minimize(mean_loss)

    # Create a session
    sess = tf.InteractiveSession()

    # Initialize the variables
    initializer = tf.global_variables_initializer()
    sess.run(initializer)

    # Choose the batch size
    batch_size = 200

    # Set early stopping mechanisms
    max_epochs = 150
    prev_validation_loss = 9999999.

    # Load the first batch of training and validation, using the class we created. 
    # Arguments are ending of 'Audiobooks_Data_<...>', where for <...> we input 'train', 'validation', or 'test'
    # depending on what we want to load
    train_data = Audiobooks_Data_Reader('train', batch_size)
    validation_data = Audiobooks_Data_Reader('validation')

    # Create the loop for epochs 
    for epoch_counter in range(max_epochs):

        # Set the epoch loss to 0, and make it a float
        curr_epoch_loss = 0.

        # Iterate over the training data 
        # Since train_data is an instance of the Audiobooks_Data_Reader class,
        # we can iterate through it by implicitly using the __next__ method we defined above.
        # As a reminder, it batches samples together, one-hot encodes the targets, and returns
        # inputs and targets batch by batch
        for input_batch, target_batch in train_data:
            _, batch_loss = sess.run([optimize, mean_loss], 
                feed_dict={inputs: input_batch, targets: target_batch})

            #Record the batch loss into the current epoch loss
            curr_epoch_loss += batch_loss

        # Find the mean curr_epoch_loss
        # batch_count is a variable, defined in the Audiobooks_Data_Reader class
        curr_epoch_loss /= train_data.batch_count

        # Set validation loss and accuracy for the epoch to zero
        validation_loss = 0.
        validation_accuracy = 0.

        # Use the same logic of the code to forward propagate the validation set
        # There will be a single batch, as the class was created in this way
        for input_batch, target_batch in validation_data:
            validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                feed_dict={inputs: input_batch, targets: target_batch})

        # Print statistics for the current epoch
        print('Epoch '+ str(epoch_counter + 1) +
              '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss) +
              '. Validation loss: '+'{0:.3f}'.format(validation_loss) +
              '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.) + '%'+
              '. Hidden_layers:' '{0}'.format(hidden_layer_size))

        # Trigger early stopping if validation loss begins increasing.
        if validation_loss > prev_validation_loss:
            break

        # Store this epoch's validation loss to be used as previous in the next iteration.
        prev_validation_loss = validation_loss

    print('End of training.')

Epoch 1. Training loss: 0.714. Validation loss: 0.676. Validation accuracy: 53.47%. Hidden_layers:700
Epoch 2. Training loss: 0.669. Validation loss: 0.651. Validation accuracy: 68.46%. Hidden_layers:700
Epoch 3. Training loss: 0.637. Validation loss: 0.605. Validation accuracy: 75.84%. Hidden_layers:700
Epoch 4. Training loss: 0.582. Validation loss: 0.537. Validation accuracy: 76.06%. Hidden_layers:700
Epoch 5. Training loss: 0.517. Validation loss: 0.475. Validation accuracy: 75.39%. Hidden_layers:700
Epoch 6. Training loss: 0.464. Validation loss: 0.435. Validation accuracy: 76.29%. Hidden_layers:700
Epoch 7. Training loss: 0.431. Validation loss: 0.412. Validation accuracy: 75.84%. Hidden_layers:700
Epoch 8. Training loss: 0.410. Validation loss: 0.397. Validation accuracy: 76.51%. Hidden_layers:700
Epoch 9. Training loss: 0.395. Validation loss: 0.387. Validation accuracy: 77.63%. Hidden_layers:700
Epoch 10. Training loss: 0.385. Validation loss: 0.380. Validation accuracy: 78.08

Epoch 81. Training loss: 0.308. Validation loss: 0.337. Validation accuracy: 80.76%. Hidden_layers:700
Epoch 82. Training loss: 0.308. Validation loss: 0.337. Validation accuracy: 80.76%. Hidden_layers:700
Epoch 83. Training loss: 0.308. Validation loss: 0.337. Validation accuracy: 80.76%. Hidden_layers:700
Epoch 84. Training loss: 0.308. Validation loss: 0.337. Validation accuracy: 80.76%. Hidden_layers:700
End of training.


### Testing 2

In [152]:
# Load the test data, following the same logic as we did for the train_data and validation data
test_data = Audiobooks_Data_Reader('test')

# Forward propagate through the training set. This time we only need the accuracy
for inputs_batch, targets_batch in test_data:
    test_accuracy = sess.run([accuracy],
                     feed_dict={inputs: inputs_batch, targets: targets_batch})

# Get the test accuracy in percentages
# When sess.run is has a single output, we get a list (that's how it was coded by Google), rather than a float.
# Therefore, we must take the first value from the list (the value at position 0)
test_accuracy_percent = test_accuracy[0] * 100.

# Print the test accuracy
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 85.04%
